In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 95456, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 95456 (delta 103), reused 136 (delta 94), pack-reused 95265
Receiving objects: 100% (95456/95456), 86.02 MiB | 27.32 MiB/s, done.
Resolving deltas: 100% (70036/70036), done.


In [ ]:
# Clone should now be in the machine
!ls

sample_data  transformers


In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

In [ ]:
!nvidia-smi

Tue May  3 10:33:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
os.chdir("/content/transformers")
os.chdir("/content/transformers/examples/tensorflow/language-modeling")
!ls

eval_tmp.txt  requirements.txt	run_mlm.py
README.md     run_clm.py	train_tmp.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
!ls

eval_tmp.txt  requirements.txt	run_mlm.py
README.md     run_clm.py	train_tmp.txt


In [ ]:
!pip install pyarrow --upgrade

In [ ]:
import os
os.chdir("/content/transformers")
os.chdir("/content/transformers/examples/tensorflow/language-modeling")
!ls

eval_tmp.txt  requirements.txt	run_mlm.py
README.md     run_clm.py	train_tmp.txt


In [ ]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-zxex5o50
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-zxex5o50
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
from sklearn.model_selection import train_test_split

with open('/content/sample_data/al-green.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]
train,eval = train_test_split(dataset, train_size=.9, random_state=2020)
print(len(train))
#print("training size:" + str(len(train)))
#print("Evaluation size: " + str(len(eval))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

181


In [ ]:
%cd /content/transformers/examples/tensorflow/language-modeling
!cp "/content/train_tmp.txt" "/content/transformers/examples/tensorflow/language-modeling/"
!cp "/content/eval_tmp.txt" "/content/transformers/examples/tensorflow/language-modeling/"

/content
cp: cannot stat '/content/train_tmp.txt': No such file or directory
cp: cannot stat '/content/eval_tmp.txt': No such file or directory


In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/sample_data/output"

Using custom data configuration default-fce3042269083628
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-fce3042269083628/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
100% 2/2 [00:00<00:00, 803.43it/s]
loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("<path to model directory>", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
input_ids = tokenizer.encode("Some text to encode", return_tensors='tf')